In [1]:
# Dependencies
import pandas as pd
import matplotlib.pyplot as plt
import json 
import sys
import os 

##Scope: 2.	Es “éxito” el que tu video aparezca más días como trending?


In [2]:
# Read original dataset. Date range varies from Nov 2017 up to June 2018
us_csv = "Resources/large_USvideos.csv"

original_df = pd.read_csv(us_csv)
original_df.head(500)

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description
0,2kyS6SvSYSE,17.14.11,WE WANT TO TALK ABOUT OUR MARRIAGE,CaseyNeistat,22,2017-11-13T17:13:01.000Z,SHANtell martin,748374,57527,2966,15954,https://i.ytimg.com/vi/2kyS6SvSYSE/default.jpg,False,False,False,SHANTELL'S CHANNEL - https://www.youtube.com/s...
1,1ZAPwfrtAFY,17.14.11,The Trump Presidency: Last Week Tonight with J...,LastWeekTonight,24,2017-11-13T07:30:00.000Z,"last week tonight trump presidency|""last week ...",2418783,97185,6146,12703,https://i.ytimg.com/vi/1ZAPwfrtAFY/default.jpg,False,False,False,"One year after the presidential election, John..."
2,5qpjK5DgCt4,17.14.11,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,23,2017-11-12T19:05:24.000Z,"racist superman|""rudy""|""mancuso""|""king""|""bach""...",3191434,146033,5339,8181,https://i.ytimg.com/vi/5qpjK5DgCt4/default.jpg,False,False,False,WATCH MY PREVIOUS VIDEO ▶ \n\nSUBSCRIBE ► http...
3,puqaWrEC7tY,17.14.11,Nickelback Lyrics: Real or Fake?,Good Mythical Morning,24,2017-11-13T11:00:04.000Z,"rhett and link|""gmm""|""good mythical morning""|""...",343168,10172,666,2146,https://i.ytimg.com/vi/puqaWrEC7tY/default.jpg,False,False,False,Today we find out if Link is a Nickelback amat...
4,d380meD0W0M,17.14.11,I Dare You: GOING BALD!?,nigahiga,24,2017-11-12T18:01:41.000Z,"ryan|""higa""|""higatv""|""nigahiga""|""i dare you""|""...",2095731,132235,1989,17518,https://i.ytimg.com/vi/d380meD0W0M/default.jpg,False,False,False,I know it's been a while since we did this sho...
5,gHZ1Qz0KiKM,17.14.11,2 Weeks with iPhone X,iJustine,28,2017-11-13T19:07:23.000Z,"ijustine|""week with iPhone X""|""iphone x""|""appl...",119180,9763,511,1434,https://i.ytimg.com/vi/gHZ1Qz0KiKM/default.jpg,False,False,False,Using the iPhone for the past two weeks -- her...
6,39idVpFF7NQ,17.14.11,Roy Moore & Jeff Sessions Cold Open - SNL,Saturday Night Live,24,2017-11-12T05:37:17.000Z,"SNL|""Saturday Night Live""|""SNL Season 43""|""Epi...",2103417,15993,2445,1970,https://i.ytimg.com/vi/39idVpFF7NQ/default.jpg,False,False,False,Embattled Alabama Senate candidate Roy Moore (...
7,nc99ccSXST0,17.14.11,5 Ice Cream Gadgets put to the Test,CrazyRussianHacker,28,2017-11-12T21:50:37.000Z,"5 Ice Cream Gadgets|""Ice Cream""|""Cream Sandwic...",817732,23663,778,3432,https://i.ytimg.com/vi/nc99ccSXST0/default.jpg,False,False,False,Ice Cream Pint Combination Lock - http://amzn....
8,jr9QtXwC9vc,17.14.11,The Greatest Showman | Official Trailer 2 [HD]...,20th Century Fox,1,2017-11-13T14:00:23.000Z,"Trailer|""Hugh Jackman""|""Michelle Williams""|""Za...",826059,3543,119,340,https://i.ytimg.com/vi/jr9QtXwC9vc/default.jpg,False,False,False,"Inspired by the imagination of P.T. Barnum, Th..."
9,TUmyygCMMGA,17.14.11,Why the rise of the robots won’t mean the end ...,Vox,25,2017-11-13T13:45:16.000Z,"vox.com|""vox""|""explain""|""shift change""|""future...",256426,12654,1363,2368,https://i.ytimg.com/vi/TUmyygCMMGA/default.jpg,False,False,False,"For now, at least, we have better things to wo..."


In [3]:
#original_df.set_index("video_id")
#original_df.loc['1ZAPwfrtAFY']

In [4]:
# Rename column name 
df= original_df.rename(columns={"video_id": "id"})

#                                        "trending_date": "Trending Date",
#                                        "title": "Title",
#                                        "channel_title": "Channel",
#                                        "category_id": "Category",
#                                        "publish_time": "Publish Date",
#                                        "tags": "Tags",
#                                        "views": "Views",
#                                        "likes": "Likes",
#                                        "dislikes": "Dislikes",
#                                        "comment_count": "Comments"})

df.columns

Index(['id', 'trending_date', 'title', 'channel_title', 'category_id',
       'publish_time', 'tags', 'views', 'likes', 'dislikes', 'comment_count',
       'thumbnail_link', 'comments_disabled', 'ratings_disabled',
       'video_error_or_removed', 'description'],
      dtype='object')

## Cleaning Data 

In [5]:
# Convert column Title to propper case  
df["title"] = df.title.str.capitalize()

# Deleting columns with no info 
del df['ratings_disabled']
del df['comments_disabled']
del df['video_error_or_removed']


In [6]:
df.head(100)

,id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,description
0,2kyS6SvSYSE,17.14.11,We want to talk about our marriage,CaseyNeistat,22,2017-11-13T17:13:01.000Z,SHANtell martin,748374,57527,2966,15954,https://i.ytimg.com/vi/2kyS6SvSYSE/default.jpg,SHANTELL'S CHANNEL - https://www.youtube.com/s...
1,1ZAPwfrtAFY,17.14.11,The trump presidency: last week tonight with j...,LastWeekTonight,24,2017-11-13T07:30:00.000Z,"last week tonight trump presidency|""last week ...",2418783,97185,6146,12703,https://i.ytimg.com/vi/1ZAPwfrtAFY/default.jpg,"One year after the presidential election, John..."
2,5qpjK5DgCt4,17.14.11,"Racist superman | rudy mancuso, king bach & le...",Rudy Mancuso,23,2017-11-12T19:05:24.000Z,"racist superman|""rudy""|""mancuso""|""king""|""bach""...",3191434,146033,5339,8181,https://i.ytimg.com/vi/5qpjK5DgCt4/default.jpg,WATCH MY PREVIOUS VIDEO ▶ \n\nSUBSCRIBE ► http...
3,puqaWrEC7tY,17.14.11,Nickelback lyrics: real or fake?,Good Mythical Morning,24,2017-11-13T11:00:04.000Z,"rhett and link|""gmm""|""good mythical morning""|""...",343168,10172,666,2146,https://i.ytimg.com/vi/puqaWrEC7tY/default.jpg,Today we find out if Link is a Nickelback amat...
4,d380meD0W0M,17.14.11,I dare you: going bald!?,nigahiga,24,2017-11-12T18:01:41.000Z,"ryan|""higa""|""higatv""|""nigahiga""|""i dare you""|""...",2095731,132235,1989,17518,https://i.ytimg.com/vi/d380meD0W0M/default.jpg,I know it's been a while since we did this sho...
5,gHZ1Qz0KiKM,17.14.11,2 weeks with iphone x,iJustine,28,2017-11-13T19:07:23.000Z,"ijustine|""week with iPhone X""|""iphone x""|""appl...",119180,9763,511,1434,https://i.ytimg.com/vi/gHZ1Qz0KiKM/default.jpg,Using the iPhone for the past two weeks -- her...
6,39idVpFF7NQ,17.14.11,Roy moore & jeff sessions cold open - snl,Saturday Night Live,24,2017-11-12T05:37:17.000Z,"SNL|""Saturday Night Live""|""SNL Season 43""|""Epi...",2103417,15993,2445,1970,https://i.ytimg.com/vi/39idVpFF7NQ/default.jpg,Embattled Alabama Senate candidate Roy Moore (...
7,nc99ccSXST0,17.14.11,5 ice cream gadgets put to the test,CrazyRussianHacker,28,2017-11-12T21:50:37.000Z,"5 Ice Cream Gadgets|""Ice Cream""|""Cream Sandwic...",817732,23663,778,3432,https://i.ytimg.com/vi/nc99ccSXST0/default.jpg,Ice Cream Pint Combination Lock - http://amzn....
8,jr9QtXwC9vc,17.14.11,The greatest showman | official trailer 2 [hd]...,20th Century Fox,1,2017-11-13T14:00:23.000Z,"Trailer|""Hugh Jackman""|""Michelle Williams""|""Za...",826059,3543,119,340,https://i.ytimg.com/vi/jr9QtXwC9vc/default.jpg,"Inspired by the imagination of P.T. Barnum, Th..."
9,TUmyygCMMGA,17.14.11,Why the rise of the robots won’t mean the end ...,Vox,25,2017-11-13T13:45:16.000Z,"vox.com|""vox""|""explain""|""shift change""|""future...",256426,12654,1363,2368,https://i.ytimg.com/vi/TUmyygCMMGA/default.jpg,"For now, at least, we have better things to wo..."


In [7]:
df["id"].count()

40949

In [8]:
# Count how many days each video was trending 
video_by_id = df.set_index("id").groupby(["id"])
video_trending_count = video_by_id.count()["title"]

video_trending_count.sort_values(ascending=False)

id
j4KvrAUjn6c    30
QBL8IRJ5yHU    29
t4pRQ0jn23Q    29
iILJvqrAQ_w    29
8h--kFui1JA    29
MAjY8mCTXWk    29
r-3iathMo7o    29
NBSAQenU2Bk    29
WIV3xNz8NoM    28
6S9c5nnDd_s    28
YI3tsmFsrOg    28
vjSohj-Iclc    28
ulNswX3If6U    28
UfKmSfgFxi8    28
2PH7dK6SLC8    28
mdWcaWBxxcY    28
6SuMbFuKDf8    27
BspHjvU11y4    27
-kTWIm7m4Uw    27
H7gh2fmdjCU    27
um-1fAVU1OQ    27
TjXQzRWmb_I    27
Xpv-sEKl1B4    27
nRc0kmOYgzQ    27
1purAy2MsOc    27
0zZ0Y_UZRBw    27
pFc6I0rgmgY    27
ju_inUnrLc4    26
ooyjaVdt-jA    26
D6Oy4LfoqsU    26
               ..
4qSziR6sD8Q     1
uEhv-4Eb9ms     1
J_fFW78fc7s     1
JBw1nwYJESc     1
ilTHiqAmuX8     1
IJKjMIU55pE     1
thIRJLsnIxY     1
tclPO3kOlfM     1
IJlXXvSFyUU     1
IKuKZIISPE0     1
jQF5Q3773uk     1
IOuFE-6Awos     1
5Smb3tZD1Qc     1
5SkwvwSydmk     1
5RjEaUqucbw     1
IXJhN4cOVM0     1
jGyUVyp9w9g     1
jE2VvGZktJo     1
J-NEOU-Ujeg     1
IbmvDn6I6sI     1
IfYRzxeMdGs     1
j28Gt5_Mlo8     1
5HUCAF6898w     1
5HOijUtExiM     1
IlkBLwl

### 2. Es Exito =  el que tu video aparezca más días como trending?

In [45]:
## Cual es la categoria que tiene mas videos trending 

df_top_freq = df.groupby(['id', 'category_id'])['id'].agg(
    {"video_count": len}).sort_values(
    "video_count", ascending=False).head(200).reset_index()
df_top_freq.head()

C:\Users\BohuCyn\Anaconda\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  after removing the cwd from sys.path.


,id,category_id,video_count
0,j4KvrAUjn6c,24,30
1,MAjY8mCTXWk,10,29
2,r-3iathMo7o,28,29
3,NBSAQenU2Bk,1,29
4,8h--kFui1JA,10,29


In [10]:
#pasar categorias a nombres 
filepath = os.path.join("Resources/US_category_id.json")
with open(filepath) as jsonfile:
    categories_data = json.load(jsonfile)

In [36]:
def video_category_name(dF): 
    for index,row in df_top_freq.iterrows():
        x = row['category_id']
        #x = category_frecuence_df.loc[index,'category_count']
        if df_top_freq.loc[index,'category_id'] == x: 
            try: 
                df_top_freq.loc[index,"category_name"] = categories_data["items"][x-1]["snippet"]["title"] 
            except: 
                pass
video_category_name(df_top_freq)
df_top_freq.sort_values("category_id",ascending=False)

,id,category_id,video_count,category_name
36,V6ElE2xs48c,43,25,Family
50,CJRw18MtRPg,28,24,Sci-Fi/Fantasy
8,vjSohj-Iclc,28,28,Sci-Fi/Fantasy
89,_1LdMWlNYS4,28,21,Sci-Fi/Fantasy
19,um-1fAVU1OQ,28,27,Sci-Fi/Fantasy
2,r-3iathMo7o,28,29,Sci-Fi/Fantasy
90,AkCLxl3MzNA,27,21,Horror
84,SQsPvrev_bQ,27,22,Horror
9,YI3tsmFsrOg,27,28,Horror
5,QBL8IRJ5yHU,26,29,Foreign


In [41]:
df_top_freq.groupby("category_name").count().sort_values("category_id",ascending=False)

,id,category_id,video_count
category_name,,,
People & Blogs,29,29,29
Drama,26,26,26
Foreign,9,9,9
Comedy,6,6,6
Documentary,6,6,6
Film & Animation,6,6,6
Sci-Fi/Fantasy,5,5,5
Action/Adventure,4,4,4
Horror,3,3,3


In [17]:
# que categoria se repite mas veces? 
category_frecuence = df_top_freq.groupby("category_id").id.nunique()
category_frecuence.sort_values(ascending=False)
category_frecuence_df = pd.DataFrame(category_frecuence) 
category_frecuence_df.sort_values("id",ascending=False)
category_frecuence_df= category_frecuence_df.rename(columns={"id": "category_count"})
category_frecuence_df.sort_values("category_count",ascending=False)

,category_count
category_id,
10,29
24,26
26,9
1,6
22,6
23,6
28,5
20,4
27,3


In [18]:
#pasar categorias a nombres 
filepath = os.path.join("Resources/US_category_id.json")
with open(filepath) as jsonfile:
    categories_data = json.load(jsonfile)
categories_data


{'kind': 'youtube#videoCategoryListResponse',
 'etag': '"m2yskBQFythfE4irbTIeOgYYfBU/S730Ilt-Fi-emsQJvJAAShlR6hM"',
 'items': [{'kind': 'youtube#videoCategory',
   'etag': '"m2yskBQFythfE4irbTIeOgYYfBU/Xy1mB4_yLrHy_BmKmPBggty2mZQ"',
   'id': '1',
   'snippet': {'channelId': 'UCBR8-60-B28hp2BmDPdntcQ',
    'title': 'Film & Animation',
    'assignable': True}},
  {'kind': 'youtube#videoCategory',
   'etag': '"m2yskBQFythfE4irbTIeOgYYfBU/UZ1oLIIz2dxIhO45ZTFR3a3NyTA"',
   'id': '2',
   'snippet': {'channelId': 'UCBR8-60-B28hp2BmDPdntcQ',
    'title': 'Autos & Vehicles',
    'assignable': True}},
  {'kind': 'youtube#videoCategory',
   'etag': '"m2yskBQFythfE4irbTIeOgYYfBU/nqRIq97-xe5XRZTxbknKFVe5Lmg"',
   'id': '10',
   'snippet': {'channelId': 'UCBR8-60-B28hp2BmDPdntcQ',
    'title': 'Music',
    'assignable': True}},
  {'kind': 'youtube#videoCategory',
   'etag': '"m2yskBQFythfE4irbTIeOgYYfBU/HwXKamM1Q20q9BN-oBJavSGkfDI"',
   'id': '15',
   'snippet': {'channelId': 'UCBR8-60-B28hp2BmDPdnt

In [19]:
category_frecuence_df["category_name"] = ""
category_frecuence_df.sort_values("category_count",ascending=False)
category_frecuence_df.reset_index()

,category_id,category_count,category_name
0,1,6,
1,10,29,
2,15,2,
3,17,1,
4,19,1,
5,20,4,
6,22,6,
7,23,6,
8,24,26,
9,25,1,


In [20]:
#puedo acer una definicion para que solo tome el numero del id y de el resto 
#!!!!!!!!!!!!!!!!!!!!! algo no esta bien pues la cat 19 es Travel & Events y me esta dando film and animation! 
# la def esta basada en el category_count' en vez del category_id pero cuando lo cambio me da problemas
# quizas es por el index ?  intente reset el index pero tampoco me funciona ??? 

def video_category_name(dF): 
    for index,row in category_frecuence_df.iterrows():
        x = row['category_count']
        #x = category_frecuence_df.loc[index,'category_count']
        if category_frecuence_df.loc[index,'category_count'] == x: 
            category_frecuence_df.loc[index,"category_name"] = categories_data["items"][x-1]["snippet"]["title"] 
video_category_name(category_frecuence_df)
category_frecuence_df.sort_values("category_count",ascending=False)

,category_count,category_name
category_id,,
10,29,Thriller
24,26,Foreign
26,9,Videoblogging
1,6,Short Movies
22,6,Short Movies
23,6,Short Movies
28,5,Sports
20,4,Pets & Animals
27,3,Music


In [ ]:
category_names = category_frecuence_df["category_name"]
category_names.head(50)

In [ ]:
category_frecuence_df.reset_index().head(100)


In [ ]:
category_names = category_frecuence_df["category_name"]
category_names

In [ ]:
category_count = category_frecuence_df["category_count"]
category_count

In [ ]:
#!!!!!!!!!!!!!!!!!!!!! necesito decirle que ponga los labels verticales y ordenarlo de mayor a menor quizas? ojo 
# si lo quiero ordenar quizas se pierda el orden de los elementos ojo pues fueron sacados de listas separadas y no de un DF

plt.bar(category_names, category_count, facecolor="red", alpha=0.75, align="center")
plt.xticks(rotation=90)

In [ ]:
# bar plot eje x categoria eje y len category_frecuence_df
#!!!!!!!!!!!!!!!!!!!!!!!!!! falta sacar nombre e la categoria/ 

# Create a bar chart based off of the group series from before
category_frecuence_chart = category_frecuence_df.plot(kind='bar')

# Set the xlabel and ylabel using class methods
category_frecuence_chart.set_xlabel("Category Name")
category_frecuence_chart.set_ylabel("Qty of Trending Videos")


plt.show()
plt.tight_layout()

## chi-suqare test

In [ ]:
# cantidad de dislikes (video controversial = +3000 dislikes)

video_by_id = df.set_index("id").groupby(["id"])
top100videos_dislikes = video_by_id.max()["dislikes"]
top100videos_dislikes_df = pd.DataFrame(top100videos_dislikes)
top100videos_dislikes_df.head()
#top100videos_dislikes_df.mean()

In [ ]:
# cantidad de views (high views = +2.000.000 views)

import scipy.stats as scs

video_by_id = df.set_index("id").groupby(["id"])
top100videos_views = video_by_id.max()["views"]

top100videos_views_df = pd.DataFrame(top100videos_views)
#top100videos_views_df.sort_values("views",ascending=False)

top100videos_views_df = top100videos_views_df.groupby(['id']).head(50)
top100videos_views_df["dislikes"] = top100videos_dislikes_df["dislikes"]
top100videos_views_df["views"] = top100videos_views_df["views"]


videos_low_views = top100videos_views_df.loc[top100videos_views_df["views"]<2000000].count()
videos_high_views = top100videos_views_df.loc[top100videos_views_df["views"]>2000000].count()
videos_controversial = top100videos_views_df.loc[top100videos_views_df["dislikes"]>3000].count()
videos_no_controversial = top100videos_views_df.loc[top100videos_views_df["dislikes"]<3000].count()
#top100videos_views_df.head()

videos_low_views_no_cont = top100videos_views_df.loc[(top100videos_views_df["views"]<2000000) & (top100videos_views_df["dislikes"]<3000)].views.count()
videos_low_views_cont = top100videos_views_df.loc[(top100videos_views_df["views"]<2000000) & (top100videos_views_df["dislikes"]>3000)].views.count()
videos_high_views_cont = top100videos_views_df.loc[(top100videos_views_df["views"]>2000000) & (top100videos_views_df["dislikes"]>3000)].views.count()
videos_high_views_no_cont = top100videos_views_df.loc[(top100videos_views_df["views"]>2000000) & (top100videos_views_df["dislikes"]<3000)].views.count()

chi = {"Low Views":[videos_low_views_cont,videos_low_views_no_cont],"High Views":[videos_high_views_cont,videos_high_views_no_cont]}
# "Status":["Controversial","No Controversial"]
chi_df = pd.DataFrame(chi)
chi_df

In [ ]:
chi_df.dtypes

In [ ]:
#Average de views del ultimo dia trending, lo estamos usando como referencia para el chi-square test
top100videos_views_df.head()

In [ ]:
top100videos_views_chart = top100videos_views_df[:5].plot(kind='bar',stacked=True)

# Set the xlabel and ylabel using class methods
top100videos_views_chart.set_xlabel("Video Id")
top100videos_views_chart.set_ylabel("Max Views")


plt.show()
plt.tight_layout()

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
from scipy.stats import chi2_contingency

class ChiSquare:
    def __init__(self, dataframe):
        self.df = dataframe
        self.p = None #P-Value
        self.chi2 = None #Chi Test Statistic
        self.dof = None
        
        self.dfObserved = None
        self.dfExpected = None
        
    def _print_chisquare_result(self, colX, alpha):
        result = ""
        if self.p<alpha:
            result="{0} is IMPORTANT for Prediction".format(colX)
        else:
            result="{0} is NOT an important predictor. (Discard {0} from model)".format(colX)

        print(result)
        
    def TestIndependence(self,colX,colY, alpha=0.05):
        X = self.df[colX].astype(str)
        Y = self.df[colY].astype(str)
        
        self.dfObserved = pd.crosstab(Y,X) 
        chi2, p, dof, expected = stats.chi2_contingency(self.dfObserved.values)
        self.p = p
        self.chi2 = chi2
        self.dof = dof 
        
        self.dfExpected = pd.DataFrame(expected, columns=self.dfObserved.columns, index = self.dfObserved.index)
        
        self._print_chisquare_result(colX,alpha)


df = chi_df

#Initialize ChiSquare Class
cT = ChiSquare(df)

#Feature Selection
testColumns = ['Low Views','High Views']
for var in testColumns:
    cT.TestIndependence(colX=var,colY="High Views" )  

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import math
import seaborn as sns
import pandas as pd
%matplotlib inline

cols = ['Low Views','High Views']

chi_df

In [ ]:
#Hypotheses 
#null hypothesis --> H0:There is no statistically significant relationship between status and the # of views per video. 
#alternative hypothesis --> H1:There is a statistically significant relationship between  status and the # of views per video.
#The H0 (Null Hypothesis): There is no relationship between variable one and variable two.
#The H1 (Alternative Hypothesis): There is a relationship between variable 1 and variable 2.
#Assumptions
#The following assumptions need to be meet in order for the results of the Chi-square test to be trusted.

#When testing the data, the cells should be counts of cases and not percentages. It is okay to convert to percentages after testing the data
#The levels (groups) of the variables being tested are mutually exclusive
#Each participant contributes to only one cell within the Chi-square table
#The groups being tested must be independent
#The value of expected cells should be greater than 5 for at least 20% of the cells

views_observed = [videos_low_views_cont, videos_low_views_no_cont, videos_high_views_cont, videos_high_views_no_cont]
views_expected = [733, 4410,169,1041]

chi2_stat, p_val, dof, ex = stats.chi2_contingency(views_observed)
p_val
stats.chisquare(views_observed, views_expected)

#pvalue = 0.0 <0.01 indicating that there’s slightly less than a 1% chance that we’d observe this distribution 
#if the dice rolls were truly randomized.

In [ ]:
from scipy.stats import chisquare
chisquare(views_observed, f_exp=None, ddof=1, axis=0)

In [ ]:
chisquare(views_observed, f_exp=views_expected)

In [ ]:
chisquare(views_observed, ddof=1)

In [ ]:
chisquare([[196,4942], [695,518]],axis=1)

In [ ]:
stats.chi2_contingency(chi_df)
#2322 = chi-square value: 
#0.0 = p-value
#1 = dof
# array: expected frecuences : all of them are greather than 5 -> then the chi2 test result can be trusted (https://pythonfordatascience.org/chi-square-test-of-independence-python/) 
# We can reject the null hypothesis as the p-value is less than 0.05.
# Thus, the results indicate that there is a relationship between views  and dislikes quantity 


In [ ]:
#Now that we know our Chi-square test of independence is significant, we want to test where the relationship is 
#between the levels of the variables. In order to do this, we need to conduct multiple 2×2 Chi-square tests using 
#the Bonferroni-adjusted p-value.By comparing multiple levels (categories) against each other, the error rate of a
#false positive compounds with each test. 
#https://pythonfordatascience.org/chi-square-test-of-independence-python/

#In our example, if we were planning on conducting all possible pairwise comparisons then the formula would be 
#0.05/6 = 0.008. Meaning, a post hoc 2×2 Chi-square test would have to have a p-value less than 0.008 to be significant.

In [ ]:
%pylab inline
import pandas as pd
import matplotlib.pyplot as plt
top100videos_views_df.sort_values("dislikes",ascending=False)

#df2 = top100videos_views_df.groupby(['views', 'dislikes'])['Name'].count().unstack('Abuse/NFF').fillna(0)
#df2[['abuse','nff']].plot(kind='bar', stacked=True)


In [ ]:
# hay que sacar 
top100videos_dislikes_df = top100videos_views_df.nlargest(10, 'dislikes')

top100videos_dislikes_chart = top100videos_dislikes_df['dislikes'][:10].plot(kind='bar',stacked=True)

top100videos_dislikes_chart.set_xlabel("Video Id")
top100videos_dislikes_chart.set_ylabel("Dislikes Count")


plt.show()
plt.tight_layout()

In [ ]:
top100videos_dislikes_df.nlargest(20, 'dislikes').plot.bar()

In [ ]:
top100videos_views_df2 = top100videos_views_df.nlargest(20, 'views').plot.bar()